# Gopigo navigation & SLAM

Using SLAM (short for Simultaneous Localization and Mapping) techniques, you will be able to execute autonomous navigation with GoPiGo3.

SLAM is a technique used in robotics to explore and map an unknown environment while estimating the pose of the robot itself. As it moves all around, it will be acquiring structured information of the surroundings by processing the raw data coming from its sensors.

For optimal and easy-to-understand coverage of the topic of SLAM, we will implement a 360º-coverage Laser Distance Sensor (LDS) in the virtual robot. 

There are low-cost versions of this sensor technology, such as EAI YDLIDAR X4 (available at https://www.robotshop.com/es/es/escaner-laser-360-ydlidar-x4.html), which is the one we will make use of in the next chapter.


### Install ROS navigation & SLAM packages
First, let's prepare your machine with the required ROS packages needed for the navigation stack (http://wiki.ros.org/navigation):

In [ ]:
sudo apt install ros-melodic-navigation

And finally the slam_gmapping package, that is already available in its binary version (https://wiki.ros.org/slam_gmapping)

In [ ]:
sudo apt-get install ros-melodic-slam-gmapping

Open the .bashrc file and verify the ROS_IP and ROS_MASTER_URI environment variables and source to the proper workspace:

source ~/rUBotCoop_LabProject/devel/setup.bash

export ROS_IP=192.168.18.83

export ROS_MASTER_URI=http://192.168.18.83:11311

## What do you need to perform robot navigation with ROS?
- Mapping: First, you need a map
- Localization: Next you need to localize the robot on that map
- Path Planning: Now you can send goal locations to the robot
- Obstacle avoidance: Finally, you need to avoid obstacles

### Navigation MAP generation
The very first thing you need in order to perform Navigation is... of course, a Map. You need a Map of the environment where you want your robot to navigate at. A Map is just a representation of an environment created from the sensor readings of the robot (for example, from the laser, among others). So just by moving the robot around the environment, you can create an awesome Map of it! In terms of ROS Navigation, this is knows as Mapping. 

### Localization the robot on that map
This means, in order to perform a proper Navigation, your robot needs to know in which position of the Map it is located and with which orientation (that is, which direction the robot is facing) at every moment. In terms of ROS Navigation, this is known as Localization. Let's see a quick demonstration of localization.

### Plan a goal location to the robot
For this, we'll need some kind of system which tells the robot WHERE to go, at first, and HOW to go there, at last. In ROS, we call this system the Path Planning. The Path Planning basically takes as input the current location of the robot and the position where the robot wants to go, and gives us as an output the best and fastest path in order to reach that point.

### Obstacle avoidance
Basically, the Obstacle Avoidance system breaks the big picture (Map) into smaller pieces, which updates in real-time using the data it's getting from the sensors. This way, it assures it won't be surprised for any sudden change in the environment, or by any obstacle that appears in the way. Use the 2D Nav Goal to send the robot to different places of the Caffe. Select a destination point that requires the robot to avoid the newly inserted object, so you can see how the path is being modified by the obstacle and how the robot is avoiding it and not colliding with it.

In order to have the ROS Navigation Stack working properly, you need to provide some data to the system , depending on which is the robot you want to use and how it is built. That is, you need to have your robot properly configured.

Robot Configuration is extremely important in all the navigation modules. For instance, in the Mapping system, if you don't tell the system WHERE does your robot have the laser mounted on, which is the laser's orientation, which is the position of the wheels in the robot, etc., it won't be able to create a good and accurate Map. And as you may already know at this point, if we don't have a good Map in ROS Navigation, we have NOTHING!



## Navigation Stack

The Navigation Stack is a set of ROS nodes and algorithms which are used to autonomously move a robot from one point to another, avoiding all obstacles the robot might find in its way. The ROS Navigation Stack comes with an implementation of several navigation related algorithms which can help you perform autonomous navigation in your mobile robots.

The Navigation Stack will take as input the current location of the robot, the desired location the robot wants to go (goal pose), the Odometry data of the Robot (wheel encoders, IMU, GPS...) and data from a sensor such as a Laser. In exchange, it will output the necessary velocity commands and send them to the mobile base in order to move the robot to the specified goal position.

Summarizing, we can say that the main objective of the Navigation Stack is to move a robot from a position A to a position B, assuring it won't crash against obstacles, or get lost in the process.

The ROS Navigation Stack is generic. That means, it can be used with almost any type of moving robot, but there are some hardware considerations that will help the whole system to perform better, so they must be considered. These are the requirements:

- The Navigation package will work better in differential drive and holonomic robots. Also, the mobile robot should be controlled by sending velocity commands in the form:
    - x, y (linear velocity)
    - z (angular velocity)
- The robot should mount a planar laser somewhere around the robot. It is used to build the map of the environment and perform localization.
- Its performance will be better for square and circular shaped mobile bases.

Following there is a figure with the basic building blocks of the Navigational stack taken from the ROS official website (http://wiki.ros.org/navigation/Tutorials/RobotSetup).

<img src="./Images/3_Nav_stack.png">

According to the shown diagram, we must provide some functional blocks in order to work and communicate with the Navigation stack. Following are brief explanations of all the blocks which need to be provided as input to the ROS Navigation stack:

- Odometry source: Odometry data of a robot gives the robot position with respect to its starting position. Main odometry sources are wheel encoders, IMU, and 2D/3D cameras (visual odometry). The odom value should publish to the Navigation stack, which has a message type of nav_msgs/ Odometry. The odom message can hold the position and the velocity of the robot.
- Sensor source: Sensors are used for two tasks in navigation: one for localizing the robot in the map (using for example the laser) and the other one to detect obstacles in the path of the robot (using the laser, sonars or point clouds).
- Sensor transforms/tf: the data captured by the different robot sensors must be referenced to a common frame of reference (usually the base_link) in order to be able to compare data coming from different sensors. The robot should publish the relationship between the main robot coordinate frame and the different sensors' frames using ROS transforms.
- Base_controller: The main function of the base controller is to convert the output of the Navigation stack, which is a Twist (geometry_msgs/Twist) message, into corresponding motor velocities for the robot.

Usually we have:
- /cmd_vel: Receives the output of the Navigation Stack and transforms the commands into motor velocities.
- /laser/scan: Provides the Laser readings to the Stack.
- /odom: Provides the Odometry readings to the Stack.
- /tf: Provides the Transformations to the Stack.

## The move_base node
This is the most important node of the Navigation Stack. It's where most of the "magic" happens.

The main function of the move_base node is to move a robot from its current position to a goal position with the help of other Navigation nodes. This node links the global planner and the local planner for the path planning, connecting to the rotate recovery package if the robot is stuck in some obstacle, and connecting global costmap and local costmap for getting the map of obstacles of the environment.

Following is the list of all the packages which are linked by the move_base node:

- global-planner
- local-planner
- rotate-recovery
- clear-costmap-recovery
- costmap-2D

Following are the other packages which are interfaced to the move_base node:

- map-server
- AMCL
- gmapping

We'll have a deeper look at all this elements in further Units. For now, just try to get a general idea of how the ROS Navigation Stack works.

## Summary 
The Navigation Stack is a set of ROS nodes an algorithms, which work together in order to move a robot from a position A to a position B, avoiding the obstacles it may find in its way. In order to do this, the Navigation Stack requires many data inputs (of different kinds). In exchange, it will give as an output the necessary command velocities in order to safely move the robot to the desired position.

<img src="./Images/3_Nav_stack2.png">

# 1. Mapping
What will you learn with this unit?

- What means Mapping in ROS Navigation
- How does ROS Mapping work
- How to configure ROS to make mapping work with almost any robot
- Different ways to build a Map

You can either give the robot a prebuilt map of the environment (in the rare case that you already have one with the proper format), or you can build one by yourself. This second option is the most frequent one. So in this Unit, we will basically show you HOW to create a map from zero.

For Mapping, you'll basically need to use 2 displays of RViz:

- LaserScan Display
- Map Display

Simultaneous Localization and Mapping (SLAM) allows the robot to build a map of the environment using the following two sources of information: 
- Robot pose estimation, coming from the internal odometry (rotary encoders) and IMU sensor data 
- Distance to objects, obstacles and walls, coming from distance sensors, the LDS in particular 

Simultaneous Localization and Mapping (SLAM). This is the name that defines the robotic problem of building a map of an unknown environment while simultaneously keeping track of the robot's location on the map that is being built. This is basically the problem that Mapping is solving. The next Unit (Localization) is also involved, but we'll get there later.

So, summarizing, we need to do SLAM in order to create a Map for the robot.

## The gmapping package 
The gmapping ROS package is an implementation of a specific SLAM algorithm called gmapping. This means that, somebody has implemented the gmapping algorithm for you to use inside ROS, without having to code it yourself. So if you use the ROS Navigation stack, you only need to know (and have to worry about) how to configure gmapping for your specific robot (which is precisely what you'll learn in this Chapter).

The gmapping package contains a ROS Node called slam_gmapping, which allows you to create a 2D map using the laser and pose data that your mobile robot is providing while moving around an environment. This node basically reads data from the laser and the transforms of the robot, and turns it into an occupancy grid map (OGM).

The generated map is published during the whole process into the /map topic, which is the reason you could see the process of building the map with Rviz (because Rviz just visualizes topics).

The /map topic uses a message type of nav_msgs/OccupancyGrid, since it is an OGM. 

Another of the packages available in the ROS Navigation Stack is the map_server package. This package provides the map_saver node, which allows us to access the map data from a ROS Service, and save it into a file.. In such a two-dimensional map, the free areas and occupied areas are drawn in different intensities of gray in 8-bit format (0-255 range).
A value of -1 is assigned to unknown areas. 

Map information is stored using two files: 
- A .pgm format file, known as portable graymap format. 
- A .yaml file containing the configuration of the map. 
See the following example of its content:

In [ ]:
image: ./test_map.pgm
resolution: 0.010000
origin: [-20.000000, -20.000000, 0.000000]
negate: 0
occupied_thresh: 0.65
free_thresh: 0.196

The YAML File generated will contain the 6 following fields:

- image: Name of the file containing the image of the generated Map.
- resolution: Resolution of the map (in meters/pixel).
- origin: Coordinates of the lower-left pixel in the map. This coordinates are given in 2D (x,y). The third value indicates the rotation. If there's no rotation, the value will be 0.
- occupied_thresh: Pixels which have a value greater than this value will be considered as a completely occupied zone.
- free_thresh: Pixels which have a value smaller than this value will be considered as a completely free zone.
- negate: Inverts the colours of the Map. By default, white means completely free and black means completely occupied.

The most interesting parameters are the last two:  
- occupied_thresh = 0.65 means that a cell is considered as occupied if its probability is above 65%. 
- free_thresh = 0.196 establishes the threshold value below which the cell is considered free, that is, 19.6%. 
Given the size in pixels of the image, it is straightforward to infer the physical dimension of the cells in the map. This value is indicated by the resolution parameter, that is, 0.01 meter/pixel.

Remember that, in order to be able to visualize the files generated through the IDE, these files must be at the /home/user/catkin_ws/src directory. The files will be initially saved in the directory where you execute the command.

### Creating a launch file for the slam_gmapping node 
The main task to create this launch file, as you may imagine, is to correctly set the parameters for the slam_gmapping node. This node is highly configurable and has lots of parameters you can change in order to improve the mapping performance. This parameters will be read from the ROS Parameter Server, and can be set either in the launch file itself or in a separated parameter files (YAML file). If you don't set some parameters, it will just take the default values. You can have a look at the complete list of parameters available for the slam_gmapping node here: http://wiki.ros.org/gmapping

Let's now check some of the most important ones:

- base_frame (default: "base_link"): Indicates the name of the frame attached to the mobile base.
- map_frame (default: "map"): Indicates the name of the frame attached to the map.
- odom_frame (default: "odom"): Indicates the name of the frame attached to the odometry system.
- map_update_interval (default: 5.0): Sets the time (in seconds) to wait until update the map.
- maxRange (float): Sets the maximum range of the laser. Set this value to something slightly higher than the real sensor's maximum range.
- maxUrange (default: 80.0): Sets the maximum usable range of the laser. The laser beams will be cropped to this value.
- minimumScore (default: 0.0): Sets the minimum score to consider a laser reading good.
- xmin (default: -100.0): Initial map size
- ymin (default: -100.0): Initial map size
- xmax (default: 100.0): Initial map size
- ymax (default: 100.0): Initial map size
- delta (default: 0.05): Sets the resolution of the map
- linearUpdate (default: 1.0): Sets the linear distance that the robot has to move in order to process a laser reading.
- angularUpdate (default: 0.5): Sets the angular distance that the robot has to move in order to process a laser reading.
- temporalUpdate (default: -1.0): Sets the time (in seconds) to wait between laser readings. If this value is set to -1.0, then this function is turned off.
- particles (default: 30): Number of particles in the filter

You can see the slam_gmapping node parameters in "gopigo3_slam.launch" file wich alls the "gmapping.launch" file:

In [ ]:
<launch>
  <!-- Arguments -->
  <arg name="set_base_frame" default="base_link"/>
  <arg name="set_odom_frame" default="odom"/>
  <arg name="set_map_frame"  default="map"/>

  <!-- Gmapping -->
  <node pkg="gmapping" type="slam_gmapping" name="slam_gmapping" output="screen">
    <param name="base_frame" value="$(arg set_base_frame)"/>
    <param name="odom_frame" value="$(arg set_odom_frame)"/>
    <param name="map_frame"  value="$(arg set_map_frame)"/>
    <param name="map_update_interval" value="2.0"/>
    <param name="maxUrange" value="3.0"/>
    <param name="sigma" value="0.05"/>
    <param name="kernelSize" value="1"/>
    <param name="lstep" value="0.05"/>
    <param name="astep" value="0.05"/>
    <param name="iterations" value="5"/>
    <param name="lsigma" value="0.075"/>
    <param name="ogain" value="3.0"/>
    <param name="lskip" value="0"/>
    <param name="minimumScore" value="50"/>
    <param name="srr" value="0.1"/>
    <param name="srt" value="0.2"/>
    <param name="str" value="0.1"/>
    <param name="stt" value="0.2"/>
    <param name="linearUpdate" value="1.0"/>
    <param name="angularUpdate" value="0.2"/>
    <param name="temporalUpdate" value="0.5"/>
    <param name="resampleThreshold" value="0.5"/>
    <param name="particles" value="100"/>
    <param name="xmin" value="-10.0"/>
    <param name="ymin" value="-10.0"/>
    <param name="xmax" value="10.0"/>
    <param name="ymax" value="10.0"/>
    <param name="delta" value="0.05"/>
    <param name="llsamplerange" value="0.01"/>
    <param name="llsamplestep" value="0.01"/>
    <param name="lasamplerange" value="0.005"/>
    <param name="lasamplestep" value="0.005"/>
  </node>
</launch>


But first, let me explain you one thing. In the gmapping_demo.launch file, the parameters where loaded in the launch file itself, as you've seen. So you changed the parameters directly in the launch file. But this is not the only way you have to load parameters. In fact, parameters are usually loaded from an external file. This file that contains the parameters is usually aa YAML file.

You can create a "gmapping_params.yaml" file with all the parameters.
The structure is: name_of_paramter: value_of_parameter

In [ ]:
roslaunch virtual_slam gopigo3_slam.launch

<img src="./Images/03_slam_map1.png">

### Manually modify the map (for convenience) 
Sometimes, the map that you create will contain stuff that you do not want to be there. For example:

- There could be people detected by the mapping process that has been included in the map. You don't want them on the map as black dots!
- There could be zones where you do not want the robot to move (for instance, avoid the robot going close to down stairs area to prevent it from falling down).

For all these cases, you can take the map generated and modify it manually, just by using an image editor. You can include forbidden areas, or delete people and other stuff included in the map.

a) Download the map file (PGM file) generated in Exercise 2.3 to your local computer

b) Open the map with your favourite image editor.

c) Edit the file in order to prevent the robot to move close to the door.

# 2. Robot Localization
What will you learn with this unit?

- What does Localization mean in ROS Navigation?
- How does Localization work?
- How do we perform Localization in ROS?

When the robot moves around a map, it needs to know which is its POSITION within the map, and which is its ORIENTATION. Determining its location and rotation (better known as the Pose of the robot) by using its sensor readings is known as Robot Localization.

You can launch RViz and add displays in order to watch the localization of the robot. You'll basically need to use 3 elements of RViz:

- LaserScan Display
- Map Display
- PoseArray Display

Visualize Pose Array (Particle Clouds)

- Click the Add button under displays and choose the PoseArray display.
- In the display properties, introduce the name of the topic where the particle cloud are being published (usually /particlecloud).
- To see the robot's position, you can choose to also add the RobotModel or TF displays.
- The Fixed Frame has to be map in order to properly visualize the particles.

In [ ]:
roslaunch virtual_slam gopigo3_navigation.launch map_file:=$HOME/gopigo_pc_ws/map_stage_2.yaml

###  Monte Carlo Localization (MCL) 
Because the robot may not always move as expected, it generates many random guesses as to where it is going to move next. These guesses are known as particles. Each particle contains a full description of a possible future pose. When the robot observes the environment it's in (via sensor readings), it discards particles that don't match with these readings, and generates more particles close to those that look more probable. This way, in the end, most of the particles will converge in the most probable pose that the robot is in. So the more you move, the more data you'll get from your sensors, hence the localization will be more precise. These particles are those arrows that you saw in RViz in the previous exercise. Amazing, right?

This is known as the Monte Carlo Localization (MCL) algorithm, or also particle filter localization.

The AMCL (Adaptive Monte Carlo Localization) package provides the amcl node, which uses the MCL system in order to track the localization of a robot moving in a 2D space. This node subscribes to the data of the laser, the laser-based map, and the transformations of the robot, and publishes its estimated position in the map. On startup, the amcl node initializes its particle filter according to the parameters provided.

So, basically, what you've done in the previous exercise was the following:

- First, you launched an amcl node using the preconfigured amcl_demo.launch file.
- Second, you set up an initial pose by using the 2D Pose Estimate tool (which published that pose to the /initialpose topic).
- Then, you started moving the robot around the room, and the amcl node began reading the data published into the laser topic (/scan), the map topic(/map), and the transform topic (/tf), and published the estimated pose where the robot was in to the /amcl_pose and the /particlecloud topics.
- Finally, via RViz, you accesed the data being published by this node into the /particlecloud topic, so you were able to vizualize it, thanks to the cloud of "arrows," which were indicating the most probable position the robot was in, and its orientation.

In order to get a proper Robot localization, we need to fullfil 3 basic requirements:

- Provide Good Laser Data
- Provide Good Odometry Data
- Provide Good Laser-Based Map Data

### Creating a launch file for the AMCL node 
This AMCL node is also highly customizable and we can configure many parameters in order to improve its performance. These parameters can be set either in the launch file itself or in a separate parameters file (YAML file). You can have a look at a complete list of all of the parameters that this node has here: http://wiki.ros.org/amcl

Let's have a look at some of the most important ones:

General Parameters 
- odom_model_type (default: "diff"): It puts the odometry model to use. It can be "diff," "omni," "diff-corrected," or "omni-corrected."
- odom_frame_id (default: "odom"): Indicates the frame associated with odometry.
- base_frame_id (default: "base_link"): Indicates the frame associated with the robot base.
- global_frame_id (default: "map"): Indicates the name of the coordinate frame published by the localization system.
- use_map_topic (default: false): Indicates if the node gets the map data from the topic or from a service call.

Filter Parameters 

These parameters will allow you to configure the way that the particle filter performs.

- min_particles (default: 100): Sets the minimum allowed number of particles for the filter.
- max_particles (default: 5000): Sets the maximum allowed number of particles for the filter.
- kld_err (default: 0.01): Sets the maximum error allowed between the true distribution and the estimated distribution.
- update_min_d (default: 0.2): Sets the linear distance (in meters) that the robot has to move in order to perform a filter update.
- update_min_a (default: π/6.0): Sets the angular distance (in radians) that the robot has to move in order to perform a filter update.
- resample_interval (default: 2): Sets the number of filter updates required before resampling.
- transform_tolerance (default: 0.1): Time (in seconds) with which to post-date the transform that is published, to indicate that this transform is valid into the future.
- gui_publish_rate (default: -1.0): Maximum rate (in Hz) at which scans and paths are published for visualization. If this value is -1.0, this function is disabled.

Laser Parameters 

These parameters will allow you to configure the way the amcl node interacts with the laser.

- laser_min_range (default: -1.0): Minimum scan range to be considered; -1.0 will cause the laser's reported minimum range to be used.
- laser_max_range (default: -1.0): Maximum scan range to be considered; -1.0 will cause the laser's reported maximum range to be used.
- laser_max_beams (default: 30): How many evenly-spaced beams in each scan to be used when updating the filter.
- laser_z_hit (default: 0.95): Mixture weight for the z_hit part of the model.
- laser_z_short (default: 0.1): Mixture weight for the z_short part of the model.
- laser_z_max (default: 0.05): Mixture weight for the z_max part of the model.
- laser_z_rand (default: 0.05): Mixture weight for the z_rand part of the model.

In [ ]:
<launch>
  <!-- Arguments -->
  <arg name="scan_topic"     default="scan"/>
  <arg name="initial_pose_x" default="0.0"/>
  <arg name="initial_pose_y" default="0.0"/>
  <arg name="initial_pose_a" default="0.0"/>

  <!-- AMCL -->
  <node pkg="amcl" type="amcl" name="amcl">

    <param name="min_particles"             value="500"/>
    <param name="max_particles"             value="3000"/>
    <param name="kld_err"                   value="0.02"/>
    <param name="update_min_d"              value="0.20"/>
    <param name="update_min_a"              value="0.20"/>
    <param name="resample_interval"         value="1"/>
    <param name="transform_tolerance"       value="0.5"/>
    <param name="recovery_alpha_slow"       value="0.00"/>
    <param name="recovery_alpha_fast"       value="0.00"/>
    <param name="initial_pose_x"            value="$(arg initial_pose_x)"/>
    <param name="initial_pose_y"            value="$(arg initial_pose_y)"/>
    <param name="initial_pose_a"            value="$(arg initial_pose_a)"/>
    <param name="gui_publish_rate"          value="50.0"/>

    <remap from="scan"                      to="$(arg scan_topic)"/>
    <param name="laser_max_range"           value="3.5"/>
    <param name="laser_max_beams"           value="180"/>
    <param name="laser_z_hit"               value="0.5"/>
    <param name="laser_z_short"             value="0.05"/>
    <param name="laser_z_max"               value="0.05"/>
    <param name="laser_z_rand"              value="0.5"/>
    <param name="laser_sigma_hit"           value="0.2"/>
    <param name="laser_lambda_short"        value="0.1"/>
    <param name="laser_likelihood_max_dist" value="2.0"/>
    <param name="laser_model_type"          value="likelihood_field"/>

    <param name="odom_model_type"           value="diff"/>
    <param name="odom_alpha1"               value="0.1"/>
    <param name="odom_alpha2"               value="0.1"/>
    <param name="odom_alpha3"               value="0.1"/>
    <param name="odom_alpha4"               value="0.1"/>
    <param name="odom_frame_id"             value="odom"/>
    <param name="base_frame_id"             value="base_link"/>
    <param name="global_frame_id"           value="map" />
  </node>
</launch>


### Summary 
In order to navigate around a map autonomously, a robot needs to be able to localize itself into the map. And this is precisely the functionality that the amcl node (of the amcl package) provides us. In order to achieve this, the amcl node uses the MCL (Monte Carlo Localization) algorithm.

You can't navigate without a map, as you learned in the previous chapter. But, surely, you can't navigate if your robot isn't able to localize itself in it either. So, the localization process is another key part of ROS Navigation.

Bascially, the amcl node takes data from the laser and the odometry of the robot, and also from the map of the environment, and outputs an estimated pose of the robot. The more the robot moves around the environment, the more data the localization system will get, so the more precise the estimated pose it returns will be.

# 3. Path Planning

What will you learn with this unit?

Visualize Path Planning in Rviz
Basic concepts of the move_base node
What is the Global Planner?
What is the Global Costmap?

For now, we've seen how to create a map of an environment, and how to localize the robot in it. So, at this point (and assuming everything went well), we have all that we need in order to perform Navigation. That is, we're now ready to plan trajectories in order to move the robot from pose A to pose B.

In this chapter, you'll learn how the Path Planning process works in ROS, and all of the elements that take place in it. But first, as we've been doing in previous chapters, let's have a look at our digital best friend, RViz.

Building the map using a Gazebo simulation involves employing the following workflow: 
- Launch the robot model within a modeled environment. 
- Launch the mapping ROS package. 
- Launch a special visualization in RViz that lets us see the areas the robot is scanning as it moves. 
- Teleoperate the robot to make it cover as much as possible of the surface of the virtual environment. 
- Once the exploration is finished, save the map, generating the two files in the formats indicated in the preceding section, that is, .pgm and .yaml.

## Navigation & SLAM with gopigo3

Once your robot has generated a map, it will use it to plan a path to a given target destination. The process of executing such a plan is called navigation, and involves the following steps: 
- Launch the robot model within the modeled environment. This step is the same as the first step in the SLAM process described earlier. 
- Provide the costmap that the robot built before. Bear in mind that the map is a characteristic of the environment, not of the robot. Hence, you can build the map with one robot and use the same map in navigation for any other robot you put in the same environment.
- Set up the navigation algorithm. We will use the Adaptive Monte Carlo Localization (AMCL) algorithm, the most common choice for effective navigation.
- Launch a RViz visualization that will let you visualize the robot in the environment and easily mark the target pose (position and orientation) that it should achieve. 
- Let the robot navigate autonomously to the target location. At this point, you can relax and enjoy watching how the GoPiGo3 drives to the indicated position while avoiding the obstacles and minimizing the distance it has to cover. 
Should you want the robot to navigate to another location, you just have to indicate it in RViz once it has reached the previous target.


Let's follow these steps to build the map of a simple Gazebo world called stage_2.world:

1. Launch the robot model within a modeled environment by running the following line of code:

In [ ]:
roslaunch virtual_slam gopigo3_world.launch world:=stage_2.world

2. Launch the SLAM mapping ROS package, including an RViz visualization that superimposes the virtual model of the robot with the actual scan data:

In [ ]:
roslaunch virtual_slam gopigo3_slam.launch

3. Teleoperate the robot to make it cover as much as possible of the surface of the current Gazebo world. Let's do this as usual with the teleoperation package:

In [ ]:
rosrun key_teleop key_teleop.py /key_vel:=/cmd_vel

As you move the robot, the LDS sensor will acquire scan data from the unknown areas, and you will receive feedback in the RViz window.

4. Once you've finished the exploration, save the map, generating two files of the formats indicated in the preceding SLAM process subsection, that is, .pgm and .yaml

In [ ]:
rosrun map_server map_saver -f ~/gopigo_pc_ws/map_stage_2

You will get two files in the root folder of your workspace: map_stage_2.pgm and map_stage_2.yaml.

Provided with the map, we are ready to perform robot navigation with the GoPiGo3.

<img src="./Images/03_slam_map1.png">

## Driving along a planned trajectory using navigation

First, close all Terminals. Then, as in the SLAM process, let's proceed step by step to perform some navigation: 
Kill the previous Gazebo process:
killall gzserver && killall gzclient

1. Launch the robot model within the modeled environment. This step is the same as the first step in the SLAM process:

In [ ]:
roslaunch virtual_slam gopigo3_world.launch world:=stage_2.world

2. Set up the navigation algorithm and launch RViz. We will use AMCL, the most common choice for effective navigation.

In this step, we also provide the costmap that the robot built before. To do this, you just have to reference the .yaml map file you created before. Make sure that the corresponding .pgm file has the same name and is placed in the same location.

In [ ]:
roslaunch virtual_slam gopigo3_navigation.launch map_file:=$HOME/gopigo_pc_ws/map_stage_2.yaml

3. The RViz window, shown in the following screenshot, lets you visualize the robot in the environment and mark the target pose (position and orientation) that it should achieve:
First of all, you have to tell the robot that this is the initial pose by pressing the 2D Pose Estimate button. Then, mark it on screen (in this particular case, it isn't necessary, since the initial pose is the same as the one the robot had when it started to build the map)

Afterward, you can press 2D Nav Goal button and set the target to the bottom-left corner by clicking the left mouse button. Release the mouse when the arrow has the desired orientation. After releasing, the robot will compute the path to follow and start navigating autonomously.

The orientation of the red arrow tells the GoPiGo3 in what direction it should stay facing once it has arrived at the target, and the curved line going from the robot to the target is the planned path. Since it has a map of the environment available, the robot is able to plan a path that avoids the obstacles.

The blue square around the robot represents the local window for obstacle avoidance planning. This is used by the Dynamic Window Approach (DWA) method, which generates a local path that efficiently evades the obstacles. The DWA method performs the calculations taking into account the robot's dynamics, in particular, its limited velocity and acceleration.

<img src="./Images/03_slam_map2.png">